## Setup

In [1]:
from dotenv import load_dotenv
import os
import json
from datetime import datetime
import time
import requests
import numpy as np
import pandas as pd
load_dotenv()


date_string =  '2023-08-29 09:30:00'
date_format = "%Y-%m-%d %H:%M:%S"
date_time_obj = datetime.strptime(date_string, date_format)


ARRIVAL_TIME = int(date_time_obj.timestamp())
# Enter work address here
WORK = '3745 N Lincoln Ave, Chicago, IL 60613'
# Add your own Google Maps API Key to a .env file
API_KEY = os.getenv('GOOGLE_API_KEY')


## Functions

In [2]:
#helpers
def get_place_id(address):
    '''
    Returns id of location based on address. 
    id is a google maps identifier for location -- some API calls need it.
    '''
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
    "input": address,
    "inputtype": "textquery",
    "key": API_KEY
    }
    
    req = requests.get(base_url, 
                       params=params)
    data = req.json()
    if data['status'] == 'OK':
        return data['candidates'][0]['place_id']
    else:
        print(f"{data['status']=}")
        return
    
def get_place_coords(address):
    '''
    Returns coords based on string address
    '''
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
    "input": address,
    "inputtype": "textquery",
    "fields": "geometry",
    "key": API_KEY
    }
    
    req = requests.get(base_url, 
                       params=params)
    data = req.json()
    if data['status'] == 'OK':
        return data['candidates'][0]['geometry']['location']
    else:
        print(f"{data['status']=}")
        return

def get_duration_est(origin, dest, mode='transit'):
    '''
    Gets estimate of duration from origin to est via transit (default).
    '''
    base_url = 'https://maps.googleapis.com/maps/api/directions/json'
    params = {
        "destination": dest,
        "origin": origin,
        "mode": mode,
        "alternatives": "true",
        "key": API_KEY
    }

    req = requests.get(base_url, 
                       params=params)
    data = req.json()
    if data['status'] == 'OK':
        if mode == 'transit':
            #get average route duration
            durs = []
            for route in data['routes']:
                durs.append(route['legs'][0]['duration']['value'])
            return np.mean(durs)/3600
        else:
            return data['routes'][0]['legs'][0]['duration']['value']/60
    else:

        print(f"{data=}\n")
        print(f"{data['status']=}")
        return
    

def closest_station(address):
    '''
    Returns closest CTA train station and walking distance to it based on string address.
    Note: "CTA train station" usually returns an actual cta train station, but sometimes the API
    returns some store that's called train station or something, so make sure you check
    the output of the station it finds.
    '''
    coords = get_place_coords(address)
    lat = coords['lat']
    lng = coords['lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        "location": str(lat)+','+str(lng),
        "keyword": "CTA train station",
        "rankby": "distance",
        "key": API_KEY
    }

    req = requests.get(base_url, 
                       params=params)
    data = req.json()
    if data['status'] == 'OK':
        #get closest station info
        #station name
        name = data['results'][0]['name']
        id = data['results'][0]['place_id']
        dur = get_duration_est('place_id:'+str(get_place_id(address)), 'place_id:'+str(id), mode='walking')
        return [name, dur]
    else:
        print(f"{data['status']=}\n{data=}")
        return
    
def closest_grocery_store(address):
    '''
    Returns closest grocery store and walking distance to it based on string address.
    Note: "grocery store" usually returns an actual grocery store, but sometimes, it returns a corner store. 
    So really, the function returns the closest convenience store/liquor store/grocery store. 
    So make sure you check the output of the store it finds.
    '''
    coords = get_place_coords(address)
    lat = coords['lat']
    lng = coords['lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        "location": str(lat)+','+str(lng),
        "keyword": "grocery store",
        "rankby": "distance",
        "key": API_KEY
    }

    req = requests.get(base_url, 
                       params=params)
    data = req.json()
    if data['status'] == 'OK':
        #get closest station info
        #station name
        name = data['results'][0]['name']
        id = data['results'][0]['place_id']
        dur = get_duration_est('place_id:'+str(get_place_id(address)), 'place_id:'+str(id), mode='walking')
        return [name, dur]
    else:
        print(f"{data['status']=}\n{data=}")
        return
    
def get_lake_dist(address):
    ''' Returns estimated walking distance to lake based on string address. 
    Works based on rough coastline estimates manually entered for Edgewater through middle of 
    Lincoln Park'''
    # Manually entered coastline coords. ROUGH ESTIMATES
    """ 
    STEP 1	 -87.65406129003956	 -87.65326528037163
    STEP 2	 -87.65088539695846	 -87.64988993827025
    STEP 3	 -87.64672789332222	 -87.64520542689931
    STEP 4	 -87.64034524670144	 -87.63911556243151
    STEP 5	 -87.63648052497483	 -87.63560217892399
    STEP 6	 -87.62617667803364	 -87.62407926150496
    """
    coords = get_place_coords(address)
    lat = coords['lat']
    lng = coords['lng']

    if lng*-1 >= 87.65326:
        lake_coords = str(lat)+','+str(-87.65326)
        #lake_coords = [lat, -87.65326]
    elif lng*-1 >= 87.64988:
        #lake_coords = [lat, -87.64988]
        lake_coords = str(lat)+','+str(-87.64988)
    elif lng*-1 >= 87.64520:
        #lake_coords = [lat, -87.64520]
        lake_coords = str(lat)+','+str(-87.64520)
    elif lng*-1 >= 87.63911:
        #lake_coords = [lat, -87.63911]
        lake_coords = str(lat)+','+str(-87.63911)
    elif lng*-1 >= 87.63560:
        #lake_coords = [lat, -87.63560]
        lake_coords = str(lat)+','+str(-87.63560)
    else:
        #lake_coords = [lat, -87.62407]
        lake_coords = str(lat)+','+str(-87.62407)

    lake_dist = get_duration_est(address, lake_coords, mode='walking')
    print(lake_dist)
    print(lake_coords)
    return lake_dist
    

def get_maps_metrics(address):
    '''
    Returns commute time to WORK global (set to Plein Air), 
    name of and walking distance to closest grocery store, 
    name of and walking distance to closest station, 
    walking distance to the lake '''
    commute_est = get_duration_est(address, WORK)
    station_est = closest_station(address)
    grocery_est = closest_grocery_store(address)
    lake_est = get_lake_dist(address)
    print(f"\n\nFINAL METRICS:\n{'--'*20}\n")
    print(f'{commute_est=}\n{station_est=}\n{grocery_est=}\n{lake_est=}')
    return [commute_est, station_est, grocery_est, lake_est]

## Run

In [3]:
address = "644 W Surf St, Chicago, IL 60657"
metrics = get_maps_metrics(address)
print('-'*40)
print(metrics)

data={'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'routes': [], 'status': 'REQUEST_DENIED'}

data['status']='REQUEST_DENIED'
data['status']='REQUEST_DENIED'


TypeError: 'NoneType' object is not subscriptable